# Check for same Queeries

In [ ]:
from pathlib import Path
list_1 = []
for path in list(Path("/content/drive/MyDrive/Eva/final_logreg_pointwise").rglob('*.csv')):
  list_1.append(str(path).split('_')[-1].split('.')[0])
list_2 = []
for path in list(Path("/content/drive/MyDrive/Eva/final_result_CC").rglob('*.csv')):
  list_2.append(str(path).split('/')[-1].split('.')[0].split('t')[-1])
set_difference_1 = set(list_1) - set(list_2)
list_difference_1 = list(set_difference_1)
print(list_difference_1)
set_difference_2 = set(list_2) - set(list_1)
list_difference_2 = list(set_difference_2)
print(list_difference_2)

['pointwise/all']
['all']


# Compute Hybrid Ranking

In [ ]:
import pandas as pd
from operator import itemgetter
# loads results of all mapproaches (first pointwise)
#all_1 = pd.read_csv(str('/content/drive/MyDrive/Eva/final_result_CC/all.csv'),sep="\t",header=None, names= ['QID', 'PID', 'Rank'])
#all_2 = pd.read_csv(str('/content/drive/MyDrive/Eva/final_logreg_pointwise/all.csv'),sep="\t",header=None, names= ['QID', 'PID', 'Rank'])
#all_3 = pd.read_csv(str('/content/drive/MyDrive/Eva/final_result_XLnet_142000/all.csv'),sep="\t",header=None, names= ['QID', 'PID', 'Rank'])
all_3 = pd.read_csv(str('/content/drive/MyDrive/Eva/final_logreg_pairwise_binary/all.csv'),sep="\t",header=None, names= ['QID', 'PID', 'Rank'])
all_2 = pd.read_csv(str('/content/drive/MyDrive/Eva/final_pairwise_sum/all.csv'),sep="\t",header=None, names= ['QID', 'PID', 'Rank'])
all_1 = pd.read_csv(str('/content/drive/MyDrive/Eva/final_result_pair_CC/all.csv'),sep="\t",header=None, names= ['QID', 'PID', 'Rank'])
finale_list = []
unique_qids = list(set(all_1['QID']))
for qid in unique_qids:
  hyb_this = []
  all_1_this_query = all_1[all_1['QID']==qid]
  all_2_this_query = all_2[all_2['QID']==qid]
  all_3_this_query = all_3[all_3['QID']==qid]
  unqiue_passages = list(set(all_1_this_query['PID']))
  for pid in unqiue_passages:
    # here the weights can be changed
    hyb_this.append([qid, pid, (0.7*int(all_1_this_query[all_1_this_query['PID']==pid]['Rank']) + 0.15 *int(all_2_this_query[all_2_this_query['PID']==pid]['Rank']) + 0.15 * int(all_3_this_query[all_3_this_query['PID']==pid]['Rank']))])
  hyb_this = sorted(hyb_this, key=itemgetter(2))
  finale_list = finale_list + hyb_this

In [ ]:
qids = [row[0] for row in finale_list]
uni_qids = list(set(qids))
for u_qid in uni_qids:
  i = 0
  for row in finale_list:
    if u_qid == row[0]:
      i = i +1
      row[2] = i


# Save Ranking

In [ ]:
import csv

with open("/content/drive/MyDrive/Eva/Hybrid/Pair_70-15-15.csv", "w", newline="", ) as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(finale_list)